In [1]:
import torch
import torch.nn as nn
import os
import re
from pathlib import Path

# import gluonnlp
import numpy as np
from scipy import stats
import torch
import torch.nn as nn 

GPUIdx = "0,1"

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]= GPUIdx

from tqdm.auto import tqdm

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [2]:
from transformers import AutoTokenizer, AutoModel
Tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
Embedding = AutoModel.from_pretrained('bert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [3]:
UNCASE = True

# FileObject = open("../../Data/DefinitionDataset/LexicoDefinition.txt", 'r', encoding='utf-8')
FileObject = open("./data/DefinitionDataset.txt", 'r', encoding='utf-8')
VocabDefExamDict = {}
MAX_SEQ_LEN = 512
pbar = tqdm(total = 1)
for line in FileObject:
    Contents = line.split(" (def.) ")
    Word = Contents[0].strip()
#     print(Word)
    if UNCASE: Word = Word.lower()
    ### Word -> Definition1 -> [ex1, ex2]
    Defs = {}
    for c in Contents[1:]:
        c = c.split(" (ex.)")
        Def = c[0].strip(); Examples = c[1].strip()
        if UNCASE:
            Def = Def.lower()
            Examples = Examples.lower()
            
        Examples = Examples.split(' | ')
        ### When don't know MAX_SEQ_LEN
#         SEQ_LEN =  max(*[len(Tokenizer.encode(e)) for e in Examples], len(Tokenizer.encode(Def)))
#         if SEQ_LEN > MAX_SEQ_LEN:
#             MAX_SEQ_LEN = SEQ_LEN
        Defs[Def] = Examples if len(Examples) else ''
    
    if Word in VocabDefExamDict.keys():
        VocabDefExamDict[Word].update(Defs)
    else:
        VocabDefExamDict[Word] = Defs
    pbar.update(1)
pbar.close()

MAX_SEQ_LEN = min(512, MAX_SEQ_LEN)
MAX_SEQ_LEN = max(256, MAX_SEQ_LEN)

print(len(VocabDefExamDict), MAX_SEQ_LEN)

  0%|          | 0/1 [00:00<?, ?it/s]

30533 512


In [4]:
PAIR_list = ["Word-Word", "Word-Def", "Word-ExamWord", "Def-ExamWord"]
PAIR_cnt = dict(zip(PAIR_list, [0]*len(PAIR_list)))

WordDef_train_src, DefExam_train_src, WordExam_train_src = [], [], []
WordWord_train_src, WordExWord_train_src, DefExWord_train_src = [], [], []
WordDef_train_tgt, DefExam_train_tgt, WordExam_train_tgt = [], [], []
WordWord_train_tgt, WordExWord_train_tgt, DefExWord_train_tgt = [], [], []

### Saved Indices
for PAIR in PAIR_list:
    pbar = tqdm(total=len(VocabDefExamDict))
    for Word in VocabDefExamDict:
        if PAIR == "Word-Word":
            WordWord_train_src.append(Word+'|'+Word)
            WordWord_train_tgt.append(Word+'|'+Word)
            PAIR_cnt[PAIR] += 1
        
        elif PAIR == "Word-Def":
            Defs = list(VocabDefExamDict[Word].keys())
            for d in Defs:
                WordDef_train_src.append(Word+'|'+Word)
                WordDef_train_tgt.append(Word+'|'+d)
                PAIR_cnt[PAIR] += 1
                
        elif PAIR == "Word-ExamWord":
            w = VocabDefExamDict[Word]
            Defs = list(w.keys())
            for d in Defs:
                for ex in w[d]:
                    if ex != '':
                        WordExWord_train_src.append(Word+'|'+Word)
                        WordExWord_train_tgt.append(Word+'|'+ex)
                        PAIR_cnt[PAIR] += 1
                        
        elif PAIR == "Def-ExamWord":
            w = VocabDefExamDict[Word]
            Defs = list(w.keys())
            for d in Defs:
                for ex in w[d]:
                    if ex != '':
                        DefExWord_train_src.append(Word+'|'+d)
                        DefExWord_train_tgt.append(Word+'|'+ex)
                        PAIR_cnt[PAIR] += 1
        else:
            print("Out-of-Bound")
            break
        ### For Debug
#         break
        pbar.update(1)
    pbar.close()

# WordDef_train_src, DefExam_train_src, WordExam_train_src = torch.tensor(WordDef_train_src), torch.tensor(DefExam_train_src), torch.tensor(WordExam_train_src)
# WordWord_train_src, WordExWord_train_src, DefExWord_train_src = torch.tensor(WordWord_train_src), torch.tensor(WordExWord_train_src), torch.tensor(DefExWord_train_src)
# WordDef_train_tgt, DefExam_train_tgt, WordExam_train_tgt = torch.tensor(WordDef_train_tgt), torch.tensor(DefExam_train_tgt), torch.tensor(WordExam_train_tgt)
# WordWord_train_tgt, WordExWord_train_tgt, DefExWord_train_tgt = torch.tensor(WordWord_train_tgt), torch.tensor(WordExWord_train_tgt), torch.tensor(DefExWord_train_tgt)

  0%|          | 0/30533 [00:00<?, ?it/s]

  0%|          | 0/30533 [00:00<?, ?it/s]

  0%|          | 0/30533 [00:00<?, ?it/s]

  0%|          | 0/30533 [00:00<?, ?it/s]

In [5]:
from torch.utils.data import Dataset, DataLoader

class DefinitionDataset(Dataset):
    def __init__(self, x=None, y=None):
        self.x = x
        self.y = y
        self.len = len(self.x)
        self.token_config = {"truncation": True, "padding": "max_length",
                            "max_length": MAX_SEQ_LEN, "return_tensors": "pt"}
    def __getitem__(self, index):
        target_w = self.x[index].split('|')[0]
        x = self.x[index].split('|')[1]
        y = self.y[index].split('|')[1]
        w_tokenized = Tokenizer(target_w, add_special_tokens=False, padding="max_length",
                                max_length=10, return_tensors="pt")
        x_tokenized = Tokenizer(x, **self.token_config)
        y_tokenized = Tokenizer(y, **self.token_config)
        return w_tokenized, x_tokenized, y_tokenized
    
    def __len__(self):
        return self.len

In [6]:
import random

Hyperparams = {
    "NumEpochs": 10,
    "BatchSize": 2**7,
#     "BatchSize": 2**5,
#     "LearningRate": 2e-3,
    "LearningRate": 1e-5,
}

train_dataset_WordWord = DefinitionDataset(x=WordWord_train_src, y=WordWord_train_tgt) if len(WordWord_train_src) else None
train_dataset_WordDef = DefinitionDataset(x=WordDef_train_src, y=WordDef_train_tgt) if len(WordDef_train_src) else None
train_dataset_WordExWord = DefinitionDataset(x=WordExWord_train_src, y=WordExWord_train_tgt) if len(WordExWord_train_src) else None
train_dataset_DefExWord = DefinitionDataset(x=DefExWord_train_src, y=DefExWord_train_tgt)  if len(DefExWord_train_src) else None

train_loader_WordWord = DataLoader(dataset=train_dataset_WordWord, batch_size=Hyperparams["BatchSize"], shuffle=True, num_workers=0)
train_loader_WordDef = DataLoader(dataset=train_dataset_WordDef, batch_size=Hyperparams["BatchSize"], shuffle=True, num_workers=0)
train_loader_WordExWord = DataLoader(dataset=train_dataset_WordExWord, batch_size=Hyperparams["BatchSize"], shuffle=True, num_workers=0)
train_loader_DefExWord = DataLoader(dataset=train_dataset_DefExWord, batch_size=Hyperparams["BatchSize"], shuffle=True, num_workers=0)

print(len(WordWord_train_src), len(WordDef_train_src), len(WordExWord_train_src), len(DefExWord_train_src))

30533 93227 1167055 1167055


In [7]:
# import gluonnlp
import copy
import random
# import gensim.downloader

crit_mean = nn.MSELoss()
crit_sum = nn.MSELoss(reduction='sum')
cos = nn.CosineSimilarity(eps=1e-6)

### Data Loading
wordsim_sim = []
file_object = open("./wordsim_data/wordsim353/wordsim353_sim_rel/wordsim_similarity_goldstandard.txt", 'r', encoding='utf-8')
for line in file_object:
    line = line.split()
    wordsim_sim.append([line[0], line[1], float(line[2])])
file_object.close()
            
wordsim_rel = []
file_object = open("./wordsim_data/wordsim353/wordsim353_sim_rel/wordsim_relatedness_goldstandard.txt", 'r', encoding='utf-8')
for line in file_object:
    line = line.split()
    wordsim_rel.append([line[0], line[1], float(line[2])])
file_object.close()

rw = []
file_object = open("./wordsim_data/rw/rw.txt", 'r', encoding='utf-8')
for line in file_object:
    line = line.split()
    rw.append([line[0], line[1], float(line[2])])
file_object.close()
            
men = []
file_object = open("./wordsim_data/men/MEN/MEN_dataset_natural_form_full", 'r', encoding='utf-8')
for line in file_object:
    line = line.split()
    men.append([line[0], line[1], float(line[2])])
file_object.close()
            
sem = []
file_object1 = open("./wordsim_data/SemEval17-Task2/test/subtask1-monolingual/data/en.test.data.txt", 'r', encoding='utf-8')
file_object2 = open("./wordsim_data/SemEval17-Task2/test/subtask1-monolingual/keys/en.test.gold.txt", 'r', encoding='utf-8')
for line1, line2 in zip(file_object1, file_object2):
    line1 = line1.split()
    line2 = line2.split()
    sem.append([line1[0], line1[1], float(line2[0])])
file_object1.close()
file_object2.close()
        
simlex = []
file_object = open("./wordsim_data/SimLex-999/SimLex-999.txt", 'r', encoding='utf-8')
file_object.readline() # skip the first line
for line in file_object:
    line = line.split()
    simlex.append([line[0], line[1], float(line[3])])
file_object.close()
            
simverb = []
file_object = open("./wordsim_data/SimVerb-3000-test.txt", 'r', encoding='utf-8')
for line in file_object:
    line = line.split()
    simverb.append([line[0], line[1], float(line[3])])
file_object.close()

In [8]:
### Utils
# import gluonnlp
import copy
import random

crit_mean = nn.MSELoss()
crit_sum = nn.MSELoss(reduction='sum')
cos = nn.CosineSimilarity(dim=1, eps=1e-5)

### Evaluator
def Evaluation(model_name, tokenizer, data):
    model_score = []; human_score = []
    model_name.eval()
    for d in data:
        w1, w2, score = d
#         print(w1, w2, score, end=' ')
        w1, w2 = w1.lower().strip(), w2.lower().strip()
        w1 = tokenizer(w1, return_tensors="pt", add_special_tokens=True)
        w2 = tokenizer(w2, return_tensors="pt", add_special_tokens=True)
        w1 = { k: v.to(device) for k, v in w1.items() }
        w2 = { k: v.to(device) for k, v in w2.items() }
        
        with torch.no_grad():
            w1 = model_name(**w1)["last_hidden_state"][:,0,:]
            w2 = model_name(**w2)["last_hidden_state"][:,0,:]
        model_score.append(cos(w1,w2).to('cpu').item())
        human_score.append(score)
    return round(stats.spearmanr(model_score, human_score)[0], 4)

def ConcatEvaluation(model_names, data):
    model_score = []; human_score = []
    for d in data:
        w1, w2, score = d
        w1 = torch.tensor([Tokenizer.encode(w1)])#.to(device)
        w2 = torch.tensor([Tokenizer.encode(w2)])#.to(device)
        with torch.no_grad():
            w1 = torch.cat((model_names[0](w1)[0][:,0,:], model_names[1](w1)[0][:,0,:]), 1)
            w2 = torch.cat((model_names[0](w2)[0][:,0,:], model_names[1](w2)[0][:,0,:]), 1)
        model_score.append(cos(w1,w2).to('cpu').item())
        human_score.append(score)
    return round(stats.spearmanr(model_score, human_score)[0], 4)
        

In [ ]:
### Training Seq
PAIR_seq = ["Word-ExamWord", "Word-Def", "Def-ExamWord", "Word-Word"]

#####
### Initialization
Embedding = AutoModel.from_pretrained('bert-base-uncased')
Embedding_ = AutoModel.from_pretrained('bert-base-uncased')
# Embedding = AutoModel.from_pretrained('./save/WordWord/3/')
# Embedding_ = AutoModel.from_pretrained('./save/WordWord/10/')
# Embedding = BertModel.from_pretrained("./save/2")
# Embedding_ = BertModel.from_pretrained("./save/WordExword/")
# Embedding = nn.DataParallel(Embedding).to(device)
# Embedding_ = nn.DataParallel(Embedding_).to(device)
#####
### Fine-tuning
# Hyperparams["LearningRate"] = 1e-4
# Hyperparams["LearningRate"] = 2e-5
# Hyperparams["LearningRate"] = 5e-6
Hyperparams["LearningRate"] = 1e-6
Hyperparams["NumEpochs"] = 10
#####

optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, Embedding.parameters()), lr=Hyperparams["LearningRate"])
pbar = tqdm(total=Hyperparams["NumEpochs"])

Embedding = nn.DataParallel(Embedding).to(device)
# Embedding_ = nn.DataParallel(Embedding).to(device)
Embedding_ = copy.deepcopy(Embedding) # Initializer
Embedding_.eval()
Extractor = torch.tensor([0]).unsqueeze(1).expand(-1,10).to(device)
MaxAvg = 0
optimizer_ww = torch.optim.Adam(filter(lambda p: p.requires_grad, Embedding.parameters()), lr=Hyperparams["LearningRate"])
optimizer_wd = torch.optim.Adam(filter(lambda p: p.requires_grad, Embedding.parameters()), lr=Hyperparams["LearningRate"])
optimizer_we = torch.optim.Adam(filter(lambda p: p.requires_grad, Embedding.parameters()), lr=Hyperparams["LearningRate"])
optimizer_de = torch.optim.Adam(filter(lambda p: p.requires_grad, Embedding.parameters()), lr=Hyperparams["LearningRate"])

for ep in range(Hyperparams["NumEpochs"]):
#     random.shuffle(PAIR_list)
    Embedding.train()
    
#     Embedding_ = copy.deepcopy(Embedding) # Copier / Epoch
#     Embedding_.eval()

    for PAIR in PAIR_seq:
#         Embedding_ = copy.deepcopy(Embedding) # PAIR Copier
#         Embedding_.eval()
        ### Renew Optimizer
        print(PAIR, "in Training")
        if PAIR == "Word-Def":
            pbar2 = tqdm(total=len(train_dataset_WordDef), leave=True)
            for i, (w, x, y) in enumerate(train_loader_WordDef):
                w = { k: v[:,0,:].to(device) for k, v in w.items() }
                x = { k: v[:,0,:].to(device) for k, v in x.items() }
                y = { k: v[:,0,:].to(device) for k, v in y.items() }
                with torch.no_grad(): OutEmb2 = Embedding_(**x)
                OutEmb1 = Embedding(**y)
                idx = (x["input_ids"] == w["input_ids"][:,0].unsqueeze(1).expand(-1,MAX_SEQ_LEN)).nonzero()
                wlen = (w["input_ids"] != Extractor).sum(dim=1)
                ### W_t - D_[CLS]
                loss = 0
                for (xx, yy), wl in zip(idx, wlen):
                    break
#                     loss += crit_sum(torch.mean(OutEmb1[0][xx,yy:yy+wl,:], dim=0), OutEmb2[0][xx,0,:])
#                     loss += crit_sum(OutEmb1[0][xx,0,:], torch.mean(OutEmb2[0][xx,yy:yy+wl,:], dim=0))
                ### W_[CLS] - D_[CLS]
                loss += crit_sum(OutEmb1[0][:,0,:], OutEmb2[0][:,0,:])
                loss /= x["input_ids"].size(0)
                optimizer_wd.zero_grad()
                if len(idx): loss.backward()
                optimizer_wd.step()
                pbar2.update(x["input_ids"].size(0))
            pbar2.close()
            
        elif PAIR == "Word-Word": # Need to be Tuned
            pbar2 = tqdm(total=len(train_dataset_WordWord), leave=True)
            for i, (w, x, y) in enumerate(train_loader_WordWord):
                w = { k: v[:,0,:].to(device) for k, v in w.items() }
                x = { k: v[:,0,:].to(device) for k, v in x.items() }
                y = { k: v[:,0,:].to(device) for k, v in y.items() }
                with torch.no_grad(): OutEmb2 = Embedding_(**y)
                OutEmb1 = Embedding(**x)
                idx = (y["input_ids"] == w["input_ids"][:,0].unsqueeze(1).expand(-1,MAX_SEQ_LEN)).nonzero()
                wlen = (w["input_ids"] != Extractor).sum(dim=1)
                ###
                loss = 0
                for (xx, yy), wl in zip(idx, wlen):
                    ### W_[CLS] - W_t / W_t - W_[CLS]
                    loss += crit_sum(OutEmb1[0][xx,0,:], torch.mean(OutEmb2[0][xx,1:1+wl,:], dim=0)) # Target
#                     loss += crit_sum(torch.mean(OutEmb1[0][xx,yy:yy+wl,:], dim=0), OutEmb2[0][xx,0,:])
                loss /= x["input_ids"].size(0)#*2
                optimizer_ww.zero_grad()
                loss.backward()
                optimizer_ww.step()
                pbar2.update(x["input_ids"].size(0))
            pbar2.close()
            
        elif PAIR == "Word-ExamWord":
            pbar2 = tqdm(total=len(train_dataset_WordExWord), leave=True)
            for i, (w, x, y) in enumerate(train_loader_WordExWord):
                w = { k: v[:,0,:].to(device) for k, v in w.items() }
                x = { k: v[:,0,:].to(device) for k, v in x.items() }
                y = { k: v[:,0,:].to(device) for k, v in y.items() }
                with torch.no_grad(): OutEmb2 = Embedding_(**y)
                OutEmb1 = Embedding(**x)
                idx = (y["input_ids"] == w["input_ids"][:,0].unsqueeze(1).expand(-1,MAX_SEQ_LEN)).nonzero()
                wlen = (w["input_ids"] != Extractor).sum(dim=1)
#                 wlen = wlen - SpTokenCnt
                loss = 0
                for (xx, yy), wl in zip(idx, wlen):
                    loss += crit_sum(OutEmb1[0][xx,0,:], torch.mean(OutEmb2[0][xx,yy:yy+wl,:], dim=0))
                loss /= x["input_ids"].size(0)
                optimizer_we.zero_grad()
                loss.backward()
                optimizer_we.step()
                pbar2.update(x["input_ids"].size(0))
            pbar2.close()
            
        elif PAIR == "Def-ExamWord":
            pbar2 = tqdm(total=len(train_dataset_DefExWord), leave=True)
            for i, (w, x, y) in enumerate(train_loader_DefExWord):
                w = { k: v[:,0,:].to(device) for k, v in w.items() }
                x = { k: v[:,0,:].to(device) for k, v in x.items() }
                y = { k: v[:,0,:].to(device) for k, v in y.items() }
                ##### Def->ExamWord
                with torch.no_grad(): OutEmb2 = Embedding_(**y)
                OutEmb1 = Embedding(**x)
                idx = (y["input_ids"] == w["input_ids"][:,0].unsqueeze(1).expand(-1,MAX_SEQ_LEN)).nonzero()
                wlen = (w["input_ids"] != Extractor).sum(dim=1)
                loss = 0
                for (xx, yy), wl in zip(idx, wlen):
                    loss += crit_sum(OutEmb1[0][xx,0,:], torch.mean(OutEmb2[0][xx,yy:yy+wl,:], dim=0))
                loss /= x["input_ids"].size(0)
                optimizer_de.zero_grad()
                if len(idx): loss.backward()
                optimizer_de.step()
                pbar2.update(x["input_ids"].size(0))
            pbar2.close()
            
                ### ExamWord->Def
#                 with torch.no_grad(): OutEmb2 = Embedding_(x)
#                 OutEmb1 = Embedding(y)
#                 idx = (y == Word[:,0].unsqueeze(1).expand(x.size(0),MAX_SEQ_LEN)).nonzero()
#                 loss = 0
#                 for xx, yy in idx:
#                     loss += crit(OutEmb1[1][xx], OutEmb2[0][xx,yy])
#                 optimizer.zero_grad()
#                 if len(idx):
# #                     loss = loss/len(idx)
#                     loss.backward()
#                 optimizer.step()
#                 pbar2.update(Hyperparams["BatchSize"])

        else:
            print("Unknown")
#             loss = crit_mean(OutEmb1[1], OutEmb2[1])

        print("WordSim Scoring")
        ScoreList = []
        ScoreList.append(Evaluation(Embedding, Tokenizer, wordsim_sim))
        ScoreList.append(Evaluation(Embedding, Tokenizer, wordsim_rel))
        ScoreList.append(Evaluation(Embedding, Tokenizer, rw))
        ScoreList.append(Evaluation(Embedding, Tokenizer, men))
        ScoreList.append(Evaluation(Embedding, Tokenizer, sem))
        ScoreList.append(Evaluation(Embedding, Tokenizer, simlex))
        ScoreList.append(Evaluation(Embedding, Tokenizer, simverb))

        print('WordSim353(Sim) Score {:.3f}'.format(ScoreList[0]))
        print('WordSim353(Rel) Score {:.3f}'.format(ScoreList[1]))
        print('RareWords Score {:.3f}'.format(ScoreList[2]))
        print('MEN Score {:.3f}'.format(ScoreList[3]))
        print('SEM Score {:.3f}'.format(ScoreList[4]))
        print('SimLex Score {:.3f}'.format(ScoreList[5]))
        print('SimVerb Score {:.3f}'.format(ScoreList[6]))
        
        Avg = sum(ScoreList)/len(ScoreList)
        print("Average {:.3f}".format(Avg))
        if Avg > MaxAvg:
            MaxAvg = Avg
            print("=====New Best=====")
            save_dir = Path('./save/'+GPUIdx)
#             save_dir = Path('./save/10,11')
            save_dir.mkdir(parents=True, exist_ok=True)
            save_dir = str(save_dir)
            print("save to", save_dir)
            Embedding.module.save_pretrained(save_dir)
            Tokenizer.save_vocabulary(save_dir)
            
            Embedding_ = copy.deepcopy(Embedding) # PAIR Copier
            Embedding_.eval()

    pbar.update(1)
    print('Epoch [{}/{}], Loss: {:.4f}'.format(ep+1, Hyperparams["NumEpochs"], loss))
#     Embedding.module.save_pretrained('./save')
    pbar2.close()
    
pbar.close()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predicti

  0%|          | 0/10 [00:00<?, ?it/s]

Word-ExamWord in Training


  0%|          | 0/1167055 [00:00<?, ?it/s]

WordSim Scoring
WordSim353(Sim) Score 0.633
WordSim353(Rel) Score 0.422
RareWords Score 0.447
MEN Score 0.728
SEM Score 0.500
SimLex Score 0.486
SimVerb Score 0.308
Average 0.503
=====New Best=====
save to save/0,1
Word-Def in Training


...

  0%|          | 0/1167055 [00:00<?, ?it/s]

WordSim Scoring
WordSim353(Sim) Score 0.584
WordSim353(Rel) Score 0.377
RareWords Score 0.401
MEN Score 0.675
SEM Score 0.460
SimLex Score 0.406
SimVerb Score 0.325
Average 0.461
Word-Word in Training


  0%|          | 0/30533 [00:00<?, ?it/s]

WordSim353(Sim) Score 0.648
WordSim353(Rel) Score 0.432
RareWords Score 0.395
MEN Score 0.705
SEM Score 0.512
SimLex Score 0.464
SimVerb Score 0.362
Average 0.503
Epoch [5/10], Loss: 9.1807
Word-ExamWord in Training


  0%|          | 0/1167055 [00:00<?, ?it/s]

In [ ]:
# PAIR_seq = ["Word-Word", "Word-Def", "Def-ExamWord", "Word-ExamWord"]